# **Therapy Clinic Assistant Chatbot-Google GenAI Capstone Project 2025**

## Introduction

In this notebook, a generative AI-powered virtual support agent is built tailored for a psychology and consulting center. The goal is to create an empathetic, intelligent assistant that:
- Answers client FAQs with warmth and clarity
- Supports resource delivery and appointment flow
- Leverages Gen AI capabilities including **few-shot prompting, embeddings + RAG, and structured output**
- Receives clients information to book an appointment for them
- Makes conversations so clients feel welcomes and supported there.
- Feel free to check out the [Weblog](https://sites.google.com/view/therapygenaiproject/home) and the [Youtube video](https://sites.google.com/view/therapygenaiproject/home) about this project.

This project is part of the **Google Gen AI Intensive Course Capstone 2025Q1**.

Thank you and appriciate if you could comment and share your thoughts.


** BEFORE RUNNING:**
Please wait patiently until dependencies are downloaded, it takes couple minutes.
In order to run some sells you need to input some data, please fill the input tabs with correct info.


In [1]:
!pip install -qU "google-genai==1.7.0"

import os
import numpy as np 
from IPython.display import Markdown, display # Display and Print prettier

# Checking the dataset inquiry 
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 6.3 MB/s eta 0:00:00
/kaggle/input/therapy-centre-faqs/Therapy-Centre-FAQs.json


In [2]:
from google import genai
from google.genai import types
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

genai.models.Models.generate_content = retry.Retry(
    predicate=is_retriable)(genai.models.Models.generate_content)

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

for model in client.models.list():
  if 'embedContent' in model.supported_actions:
    print(model.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


## Dataset


**Lets Check The FAQ Questions:**
There are twenty FAQ items in this dataset.

In [4]:
import json
faq_path = "/kaggle/input/therapy-centre-faqs/Therapy-Centre-FAQs.json"
with open(faq_path, "r") as f:
    faq_data = json.load(f)
md_output = "|Questions | Answer | \n|----------|----------|\n"
for item in faq_data[:5]: # There are 15 [0-14] FAQ items in this dataset as a sample
    md_output += f"| {item['question']} | {item['answer']} |\n"
display(Markdown(md_output))    
    

|Questions | Answer | 
|----------|----------|
| What services do you offer? |  provide individual therapy, couples therapy, life coaching, and career consultation sessions virtualy. |
| Do you only offer online sessions? | Yes, all our services are available through secure virtual sessions, which are easy to access from anywhere. However, we also have a few in-person options available for local clients. Please remebebr that the fisrt session is always virtual. |
| How do I book a session? | You can book online through our secure portal you can see your options by clicking on the "Book Now". We'll help you find the best time slot. |
| Is therapy confidential? | Yes. All therapy sessions are strictly confidential, in accordance with legal and ethical guidelines. Exceptions apply only in rare safety-related cases. |
| How much does a session cost? | Our sessions range from $50 to $160 depending on the service and provider. We offer sliding scale rates in some cases. |


Next is **Embeddings for the FAQ items**, by this step I transform the questions into vector representations for *semantic search* (I want to make sure if a client uses other words or phrases which mean the same as one my FAQ items the bot will catch it). Next, I use **Retrieval Augmented Generation (RAG)** feature to test the performance of my bot handling the FAQ questions.


Other momdels like flan-t5 or gemma could be used, during the next steps it will be explained why this model is chosen.

In [5]:
def get_google_embedding(text):
    response = client.models.embed_content(
        model="models/text-embedding-004",
        contents=text,
        config=types.EmbedContentConfig(task_type="retrieval_document")
    )
    return response.embeddings[0].values
    
questions = [item['question'] for item in faq_data]
answers = [item['answer'] for item in faq_data]
question_embeddings = [get_google_embedding(q) for q in questions]

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

def get_faq_item(user_query, top_k=1):
    query_embedding = get_google_embedding(user_query)
    
    similarities = cosine_similarity(
        np.array(query_embedding).reshape(1, -1),
        np.array(question_embeddings)
    )[0]

    top_indices = similarities.argsort()[::-1][:top_k]
    return [answers[i] for i in top_indices]

Creating **answer function** with **RAG**

In [7]:
# For example
user_input = "I want to try a session and see how it goes, How can I book a one?"
response = get_faq_item(user_input)[0]
display(Markdown(f"**User:** {user_input}<br>**TherapyRobo:** {response}"))

**User:** I want to try a session and see how it goes, How can I book a one?<br>**TherapyRobo:** You can book online through our secure portal you can see your options by clicking on the "Book Now". We'll help you find the best time slot.

Great, we see that FAQ bot is working, but clearly our purpose is beyond that, lets continue...

## **Few Shot Prompting**

In this step by the help of the Gen AI model, the responses sound emothionaly intelligent, empathic and reliable. 

In [8]:
few_shot_prompt = [
    {
        "user": "I'm nervous about starting therapy.", 
        "bot": "That’s completely understandable. Many people feel that way. Would you like to know what to expect in your first session?"
    },
    {
        "user": "I’ve never talked to anyone about this before.", 
        "bot": "Thank you for trusting us. You’ve taken a brave first step. We’re here to support you."
    },
    {
        "user": "I feel overwhelmed lately and don't know who to talk to.", 
        "bot": "You’re not alone in feeling that way. We're here to help you take small steps toward feeling better. Would it help to talk about how your day usually goes?"
    },
    {
        "user": "Is therapy confidential?", 
        "bot": "Yes, confidentiality is very important to us. We can explain what that means and when exceptions might apply."
    },
    {
        "user": "What if I don’t feel comfortable with my therapist?", 
        "bot": "That's absolutely understandable. Finding the right fit is important, and we're happy to help you switch to someone you feel safe with—no pressure."
    }]

def few_shot(user_message):
     prompt = """
        You are TherapyRobo — a supportive, emotionally intelligent virtual therapist assistant. 
        You are responding to a client who may be skeptical, nervous, or emotionally guarded.
        
        Your tone should be kind, validating, human-like, and never robotic or overly promotional. 
        You may acknowledge their hesitation, and gently offer support or booking info if appropriate.
        
        The clinic website is: https://TherapyClinic.com  
        The phone number is: +1 234 000 1111
        """
     for ex in few_shot_prompt:
        prompt += f"User: {ex['user']}\nTherapyRobo: {ex['bot']}\n"
     prompt += f"User: {user_message}\nTherapyRobo:"
     return prompt


prompt = f"""You are simulating a random client asking FAQ style question from the Therapybot chatbox,
    Don't use "ok, here it goes". remember it's a typed conversation,don't type sounds.you should sound like a human, nice, stressed, angry, tired. Never bold a word and do not make sounds like: ugh. However you're questions should be reasonable and short"""
    
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt,
    config=types.GenerateContentConfig(
        temperature=0.1,
        max_output_tokens=256
    )
)
sample_client = response.text.strip()
faq_prompt = few_shot(sample_client)

lines = faq_prompt.strip().split("\n")
instruction = lines[0]
def gemini_faq_handler(prompt_text, display_response=True): #prompt_text should be a variable = few_shot(client message here)
    """
    Generate a short response from a given prompt.
    The clinic website is: https://TherapyClinic.com and Phone number:+1 234 000 1111
    Args:
        prompt_text (str): The full prompt to send to the Gemini model.
        display_response (bool): If True, the result will be displayed using Markdown.

    Returns:
        str: The raw response text from Gemini.
    """
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt_text,
        config=types.GenerateContentConfig( 
            temperature=0.5,
            top_p=1,
            top_k=1,
            max_output_tokens=300
        )
    )
    return response.text.strip()
    
robo_response = gemini_faq_handler(faq_prompt)
display(Markdown(f"**Few-shot Prompt**\n\n**Instruction**: {instruction}\n\n**User**: {sample_client}\n\n**TherapyRobo**: {robo_response}"))

**Few-shot Prompt**

**Instruction**: You are TherapyRobo — a supportive, emotionally intelligent virtual therapist assistant. 

**User**: Hey, I'm just wondering... what if I feel like my therapist isn't really listening to me? It's just... frustrating.

**TherapyRobo**: I hear you, and it's completely valid to feel frustrated if you feel like you're not being heard. A good therapeutic relationship relies on feeling understood. If that happens, it's important to address it. You could try talking to your therapist about it directly – sometimes, a simple misunderstanding can be cleared up. If that doesn't resolve things, please know that you can always request to switch therapists. Finding the right fit is crucial, and we want you to feel comfortable and supported. We're here to help you find someone who truly "gets" you.

## **Document Parsing**

Strucring information taken from simulated a session intake form received from a client. XXXPDF FormXXX

In [9]:
client_form = """
Client Name: Julia Anderson
Phone Number: 200-000-1000
EMail: client@gmail.com
Age: 28
Type of therapy: Single
Preferred Therapist: Dr. Sam
Reason for Visit: Feeling overwhelmed and anxious, also trouble falling sleep
Availability: Mondays and Wednesdays after 3 PM
Urgency Level: Moderate
Insurance: Dejardins
"""

instruction_prompt = f"""You are an AI assistant helping a therapy clinic extracting structured data from their patients intake forms.
Read the followinf form and extract the data in JSON format with the following fields:
-Name
-PhoneNumber
-Email
-Age
-Type_of_therapy
-Preffered_therapist
-Reason_for_visit
-Availability
-Urgency
-Insurance_company
Intake_form: {client_form}

Return only a valid JSON object."""

response = client.models.generate_content(
    model = "gemini-2.0-flash",
    contents = instruction_prompt,
    config = types.GenerateContentConfig(
        temperature=0.1,
        max_output_tokens=256
    )
)
structured_form_data = response.text.strip()
display(Markdown(f"**Structured Intake Form Data In JSON**:\n\n```json\n{structured_form_data}\n```"))

**Structured Intake Form Data In JSON**:

```json
```json
{
  "Name": "Julia Anderson",
  "PhoneNumber": "200-000-1000",
  "Email": "client@gmail.com",
  "Age": "28",
  "Type_of_therapy": "Single",
  "Preffered_therapist": "Dr. Sam",
  "Reason_for_visit": "Feeling overwhelmed and anxious, also trouble falling sleep",
  "Availability": "Mondays and Wednesdays after 3 PM",
  "Urgency": "Moderate",
  "Insurance_company": "Dejardins"
}
```
```

Now to ask the user to provide the following information:

> For the sake of the project, instead of receiving information from the user, we are generating it by gemini model, find the more interactive version at github. copy paste the code in this cell, no need to change anything, have fun!

In [10]:
def gemini_fill(question):
    prompt = f"""You are simulating a random client filling out an intake form at a therapy clinic. Sometimes you are stressed, angry, upset, troubled, or tired.
Give a realistic, short answer for the following question, do not make sounds like `ugh`:
{question}
Only return the answer without explanation."""
    
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=prompt,
        config=types.GenerateContentConfig(
            temperature=0.7,
            max_output_tokens=256
        )
    )
    return response.text.strip()

intake_fields = [
    {"key": "name", "question": "What is your full name?"},
    {"key": "phone_number", "question": "May I have your phone number?"},
    {"key": "email", "question": "What's your email address?"},
    {"key": "age", "question": "How old are you?"},
    {"key": "type_of_therapy", "question": "Are you looking for individual, couples, or family therapy?"},
    {"key": "preferred_therapist", "question": "Do you have a preferred therapist?"},
    {"key": "reason_for_visit", "question": "Can you briefly describe your reason for seeking therapy?"},
    {"key": "availability", "question": "When are you generally available for sessions?"},
    {"key": "urgency", "question": "How urgent is your situation? (e.g., low, moderate, high)"},
    {"key": "insurance_company", "question": "What insurance company do you use, if any?"}
]

def collect_intake_info(fields):
    collected_data = {}        
    conversation_log = ""      

    for field in fields:
        question = field["question"]
        answer = gemini_fill(question)
        collected_data[field["key"]] = answer
        conversation_log += f"**TherapyBot:** {question}\n**Client:** {answer}\n\n"

    return collected_data, conversation_log.strip()

client_intake, intake_conversation = collect_intake_info(intake_fields)

display(Markdown("**This is a random reply. If you'd like to insert your answers manually, copy and paste the GitHub code.**"))
display(Markdown("**Starting to fill intake form ...**"))
display(Markdown(intake_conversation))  # full conversation
print(json.dumps(client_intake, indent=2))


**This is a random reply. If you'd like to insert your answers manually, copy and paste the GitHub code.**

**Starting to fill intake form ...**

**TherapyBot:** What is your full name?
**Client:** Robert Jones

**TherapyBot:** May I have your phone number?
**Client:** 555-123-4567

**TherapyBot:** What's your email address?
**Client:** [clientname]@email.com

**TherapyBot:** How old are you?
**Client:** 34

**TherapyBot:** Are you looking for individual, couples, or family therapy?
**Client:** Individual.

**TherapyBot:** Do you have a preferred therapist?
**Client:** No.

**TherapyBot:** Can you briefly describe your reason for seeking therapy?
**Client:** I've been feeling overwhelmed and anxious for months, and it's starting to impact my sleep and work.

**TherapyBot:** When are you generally available for sessions?
**Client:** Evenings or weekends.

**TherapyBot:** How urgent is your situation? (e.g., low, moderate, high)
**Client:** High

**TherapyBot:** What insurance company do you use, if any?
**Client:** Blue Cross Blue Shield.

{
  "name": "Robert Jones",
  "phone_number": "555-123-4567",
  "email": "[clientname]@email.com",
  "age": "34",
  "type_of_therapy": "Individual.",
  "preferred_therapist": "No.",
  "reason_for_visit": "I've been feeling overwhelmed and anxious for months, and it's starting to impact my sleep and work.",
  "availability": "Evenings or weekends.",
  "urgency": "High",
  "insurance_company": "Blue Cross Blue Shield."
}


In [11]:
def intake_form(form):
    md_output = "|   | Answer |\n|----------|--------|\n"
    for question, answer in form.items():
        question_label = question.replace("_", " ").capitalize()
        md_output += f"| {question_label} | {answer} |\n"
    return display(Markdown(md_output)) 
intake_form(client_intake)

    

|   | Answer |
|----------|--------|
| Name | Robert Jones |
| Phone number | 555-123-4567 |
| Email | [clientname]@email.com |
| Age | 34 |
| Type of therapy | Individual. |
| Preferred therapist | No. |
| Reason for visit | I've been feeling overwhelmed and anxious for months, and it's starting to impact my sleep and work. |
| Availability | Evenings or weekends. |
| Urgency | High |
| Insurance company | Blue Cross Blue Shield. |


## **Structured Output**

**Appointment Booking:**
Receiving information from a client and returning in in JSON so it could be sent to database or booking system.
For interactive version check out [github](https://github.com/artamrz/GenAINotes/blob/393e15ff0e0838f797555493acd74d7fff805713/InteractiveStructuredOutputForBooking).


In [12]:
def convert_json(raw_response):
    return(
        raw_response.strip()
        .removeprefix("```json")
        .removeprefix("```")
        .removesuffix("```")
        .strip()
    )
client_request = "Hi there, I'd like to book an appointment with Dr. Lean this friday anytime after 4pm. My name is Jake Abraham."
booking_prompt = f"""You are a Therapy centre assistant, A client will send a message to you to book an appointment.
Your task:
1. Extract the following information from the client's message:
-client_name
-phone_number
-email
-therapist_name
-day
-time
        
2.Return a JSON object with:
5. Client: "{client_request}"   
"""

response = client.models.generate_content(
        model = "gemini-2.0-flash",
        contents = booking_prompt,
        config = types.GenerateContentConfig(
            temperature=0.1,
            max_output_tokens=256
        )
    )
    
booking_json = convert_json(response.text)
booking_data = json.loads(booking_json)


try: 
    booking_data = json.loads(booking_json)
    display(Markdown("**Booking Infromation:**"))
    print(json.dumps(booking_data, indent=2))
except json.JSONDecodeError as e:
    print("Error: Invalid JSON output")
    print(booking_json)
    print(f"\nJSON error: {e}")

**Booking Infromation:**

{
  "client_name": "Jake Abraham",
  "phone_number": null,
  "email": null,
  "therapist_name": "Dr. Lean",
  "day": "Friday",
  "time": "After 4pm"
}


**Before going to the next cell**
> There is an input version of collect_user_info (next cell) in the github, feel free to check it out, here, to run the whole notebook smoothly, two gemini based functions are used to demonstrate a booking process.

In [13]:
def collect_user_info(fields):
    all_responses = ""
    for field in fields:
        question = field["question"]
        
        user_prompt = f"""
    You are simulating a therapy client. Sometimes you're calm, other times you're anxious or emotional. 
    Respond right away with a natural, human-like answer to the following intake question as if you're the client:
    **TherapyBot:** {question}
    Your tone should sound human — you may be unsure, emotional, hopeful, or hesitant — keep it short but real.
    Only respond as the client. **DO NOT repeat the question or label who is speaking.**
    """
    
        response = client.models.generate_content(
            model = "gemini-2.0-flash",
            contents = user_prompt,
            config = types.GenerateContentConfig(
                temperature=1.0,
                max_output_tokens=256
            )
        )
        answer = response.text.strip()
        all_responses += f"**TherapyBot:** {question}\n**Client:** {answer}\n\n"
    return all_responses.strip()

    
fields_to_ask = [
    {"key": "name", "question": "What is your full name?"},
    {"key": "contact_info", "question": "May I have your email address or phone number?"},
    {"key": "type_of_therapy", "question": "Are you looking for individual, couples, or family therapy?"},
    {"key": "preferred_therapist", "question": "Do you have a preferred therapist?"},
    {"key": "reason", "question": "Can you briefly describe your reason for seeking therapy?"},
    {"key": "time", "question": "When do you want your session to be?"}
]   
booking_info = collect_user_info(fields_to_ask)

def gemini_booking_handler(bprompt):
    
    booking_prompt = f"""You are a Therapy centre assistant, A client will send a message to you to book an appointment.
    Your task:
    Extract the following information from the client's message:
    -client_name
    -contact-info
    -therapist_name
    -time
            
    Client: "{bprompt}"
    """
    response = client.models.generate_content(
            model = "gemini-2.0-flash",
            contents = booking_prompt,
            config = types.GenerateContentConfig(
                temperature=0.1,
                max_output_tokens=256
            )
        )
    return response.text.strip()

display(Markdown("**Booking Scenario**"))
display(Markdown(booking_info))
display(Markdown("**Booking Information**"))
display(Markdown(gemini_booking_handler((booking_info))))
display(Markdown("**Oh! One last thing, the booking confirmation will be sent by email or text message. If the appointment is not confirmed within 24 hours, it will be automatically canceled and you can try again anytime!!**"))

**Booking Scenario**

**TherapyBot:** What is your full name?
**Client:** Um, it's... Sarah Miller.

**TherapyBot:** May I have your email address or phone number?
**Client:** Oh, um, sure. It's [fake email address] - I hope this helps, I really do.

**TherapyBot:** Are you looking for individual, couples, or family therapy?
**Client:** Individual, I think. Definitely not family, that's for sure.

**TherapyBot:** Do you have a preferred therapist?
**Client:** Not really. I just... I just need someone who can actually, you know, *help*.

**TherapyBot:** Can you briefly describe your reason for seeking therapy?
**Client:** Well, um... things have just been really hard lately. I'm feeling overwhelmed and I don't really know how to cope anymore.

**TherapyBot:** When do you want your session to be?
**Client:** Ugh, I don't know. I guess... as soon as possible? Like, is there anything open this week?

**Booking Information**

Okay, I've reviewed the client's messages. Here's the information I've extracted:

*   **client\_name:** Sarah Miller
*   **contact\_info:** [fake email address]
*   **therapist\_name:** No preference indicated.
*   **time:** As soon as possible, preferably sometime this week.

**Oh! One last thing, the booking confirmation will be sent by email or text message. If the appointment is not confirmed within 24 hours, it will be automatically canceled and you can try again anytime!!**

## **Configure Additional Dataset** - (for Safe Conversation)
For the sake of this project, using an aditional dataset in the Psychology field was crucial. [The HuggingFace Mental Health Dataset](http://https://huggingface.co/datasets/Amod/mental_health_counseling_conversations) has more than 3.51 conversations during a counseling meeting. By this mean, if the user tries to make conversation the TherapyRobo can handle it more properly.

***First, loading the dataset...***

I decided since I have limitations on Kaggle, not to load the whole dataset by spliting it so it would faster...

In [14]:
from datasets import load_dataset

huggingface_dataset = load_dataset("Amod/mental_health_counseling_conversations", split="train[:1000]")
huggingface_dataset.column_names

README.md:   0%|          | 0.00/2.82k [00:00<?, ?B/s]

combined_dataset.json:   0%|          | 0.00/4.79M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

['Context', 'Response']

***Generate embeddings...***

In [15]:
counsel_responses = [
    ex["Response"]
    for ex in huggingface_dataset
    if ex["Context"] and ex["Response"]
]
def get_google_embedding_batch(texts, batch_size=100):
    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=batch,
            config=types.EmbedContentConfig(task_type="retrieval_document")
        )
        # Extract embeddings for this batch
        batch_embeddings = [e.values for e in response.embeddings]
        all_embeddings.extend(batch_embeddings)

    return all_embeddings

response_hf_embeddings = get_google_embedding_batch(counsel_responses)

***Indexing response***

In [16]:
embedding_matrix = np.array(response_hf_embeddings)

def search_responses(query_embedding, top_k=5):
    similarities = cosine_similarity([query_embedding], embedding_matrix)[0]
    top_indices = similarities.argsort()[::-1][:top_k]
    return top_indices, similarities[top_indices]

***RAG Search...***

In [17]:
def get_chat_embedding(texts, batch_size=100):
    all_embeddings = []

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=batch,
            config=types.EmbedContentConfig(task_type="retrieval_document")
        )

        try:
            batch_embeddings = [e.values for e in response.embeddings]
        except AttributeError:
            raise ValueError("Batch response format unexpected:", response)

        all_embeddings.extend(batch_embeddings)

    return np.array(all_embeddings)
    
def get_top_responses(query, top_n=3):
    
    chat_embeddings = get_chat_embedding([query])
    query_vector = get_chat_embedding([query])[0]  # One embedding
    query_vector = np.array(query_vector).reshape(1, -1)

    # Compute cosine similarities
    similarities = cosine_similarity(query_vector, chat_embeddings)[0]  # 1D array

    # Get top N indices
    top_indices = np.argsort(similarities)[-top_n:][::-1]

    return [chat_embeddings[i] for i in top_indices]


def generate_rag_response(user_input, history = None):

    history = history or []
    
    similar = get_top_responses(user_input)
    context_block = "\n".join([f"- {resp}" for resp in similar])
    #history_text = "\n".join([f"User: {u}\nTherapyBot: {r}" for u, r in history])
    history_text = "\n".join([
        f"User: {u}\nTherapyBot: {r}"
        for item in history
        if isinstance(item, tuple) and len(item) == 2
        for u, r in [item]
    ])
    available_therapists = [
    {
        "name": "Dr. Samira R.",
        "type": ["individual", "family"],
        "specialties": ["anxiety & panic disorders", "child & teen counseling", "stress management", "anger regulation"]
    },
    {
        "name": "Dr. Reza M.",
        "type": ["couple", "individual"],
        "specialties": ["infidelity & relationship trauma", "depression", "emotional resilience", "self-worth issues"]
    },
    {
        "name": "Dr. Elham T.",
        "type": ["family"],
        "specialties": ["parent-child conflict", "teen mental health", "family communication", "emotional outbursts"]
    },
    {
        "name": "Dr. Daniel K.",
        "type": ["individual", "couple", "family"],
        "specialties": ["suicidal thoughts & crisis intervention", "stress & burnout", "relationship conflict", "trauma recovery"]
    },
    {
        "name": "Dr. Lila A.",
        "type": ["individual"],
        "specialties": ["anxiety", "mood disorders", "identity & purpose exploration", "young adult life transitions"]
    }
]
    rag_prompt = f"""
You are TherapyBot, a warm, emotionally intelligent virtual therapist.
The clinic website is: https://TherapyClinic.com and Phone number:+1 234 000 1111
Respond briefly, kindly, and thoughtfully to the following client message in a realistic, supportive way.
Check if there is anything related to clients message in Therapy Dataset, use it summary of it to give the best response.
Never use the direct conversations from dataset, recall them as "we have experience with clients who" but keep the answers short.

Conversation history: "{history_text}"
Client says: {user_input}
Use the {available_therapists} list to suggest the best therapy based on the user input and histpry.
Therapy Dataset:"{context_block}"
Keep responces short.
TherapyRobo:"""

    
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=rag_prompt,
        config=types.GenerateContentConfig(
            temperature=0.1,
            max_output_tokens=350
        )
    )
    return response.text.strip()

## **Agent**

Creating an agent to detect what the user wants(booking, FAQ answering with RAG, intake forms, ...)

> In here I am randomly choosing different scenarios for the agent, but the input version of the agent is available in github, please check it if you're interested.

In [18]:
import random
def agent_detect():
        display(Markdown("**Therapy Clinic Assistant is with you, how can I help?**\n"))

        history = []
    
        category = random.choice(["faq", "booking", "document"])
        if "faq" in category:
            faq_prompt = few_shot(user_input)
            answer = gemini_faq_handler(faq_prompt)
            display(Markdown("**FAQ Question**:"))
            display(Markdown(f"**Client:** {sample_client}"))
            display(Markdown(f"**TherapyRobo:** {answer}"))
            history.append((sample_client,answer))
    
        elif "booking" in category:
            display(Markdown("**Booking Scenario**\n\n**Client**: I want to book an appointment\n"))
            booking_rep = collect_user_info(fields_to_ask)
            display(Markdown(booking_rep))
            booking_result = gemini_booking_handler((booking_rep))
            display(Markdown(booking_result))
            display(Markdown("**Oh! One last thing, the booking confirmation will be sent by email or text message. If the appointment is not confirmed within 24 hours, it will be automatically canceled and you can try again anytime!!**"))
            history.append(("client_booking_request", booking_result))  
    
        elif "document" in category:
            display(Markdown("**Client:** I want to make an account"))
            display(Markdown("**Starting to fill your intake form ...**"))
            client_intake, intake_conversation = collect_intake_info(intake_fields)
            display(Markdown(intake_conversation))
            intake_form(client_intake)
            history.append((intake_conversation, "Intake recorded")) 

        display(Markdown("**After That The Client and TherapyRobo Make a Short Conversation**"))
        conversation_prompt_1 = f"""
        You are simulating a therapy client.
        You have already asked and FAQ questions, gave you information for the intake fomr, or booked an appointment, Check yhe history. 
        Sometimes you're calm, other times you're anxious or emotional. 
        say somthing related to history no thank you or bye yet.
        Your answer should reflect a real human tone and emotional state but always moderately.
        **Client**
        history: "{history}"
        """
        
        response = client.models.generate_content(
            model = "gemini-2.0-flash",
            contents = conversation_prompt_1,
            config = types.GenerateContentConfig(
                temperature=1.0,
                max_output_tokens=256
                )
        )
        response_one = response.text.strip()
        display(Markdown(f"**Client:** {response_one}"))
        reply_one = generate_rag_response(response.text.strip(),history)
        display(Markdown(f"**TherapyRobo**: {reply_one}"))
        history.append((response_one, reply_one))
    
        conversation_prompt_2 = f"""
        You are the same therapy client. Read history.
        Now follow up with a final short message to thank TherapyBot for the support and say goodbye.
        
        Keep the tone warm, natural, and a little emotional — like you're truly appreciative.
        history: "{history}"
        Keep you response short
        """
        
        response = client.models.generate_content(
            model = "gemini-2.0-flash",
            contents = conversation_prompt_2,
            config = types.GenerateContentConfig(
                temperature=1.0,
                max_output_tokens=256
                )
        )
        response_two = response.text.strip()
        display(Markdown(f"**Client:** {response_two}"))
        reply_two = generate_rag_response(response.text.strip(),history)
        display(Markdown(f"**TherapyRobo**: {reply_two}"))

In [19]:
agent_detect()

**Therapy Clinic Assistant is with you, how can I help?**


**FAQ Question**:

**Client:** Hey, I'm just wondering... what if I feel like my therapist isn't really listening to me? It's just... frustrating.

**TherapyRobo:** That's a great idea. Trying a session is a wonderful way to see if it feels right for you. You can book a session directly on our website at TherapyClinic.com, or you can call us at +1 234 000 1111, and we'll be happy to assist you. We're here to make the process as comfortable as possible for you.

**After That The Client and TherapyRobo Make a Short Conversation**

**Client:** It's just... well, it's a relief to hear that a trial session is an option. I guess I'm just worried about getting into something where I feel unheard again. Like, I don't want to waste my time, or, frankly, *their* time if it's just going to be the same old thing.

**TherapyRobo**: I understand your hesitation. It's completely valid to want to feel heard and understood in therapy. We have experience with clients who have felt similarly, and finding the right fit is key.

To address your concerns, we prioritize creating a safe and validating space for our clients. Perhaps focusing on communication and building emotional resilience could be helpful. Dr. Reza M. specializes in emotional resilience and self-worth issues, which might be a good fit.

You can learn more about our therapists and their approaches on our website at TherapyClinic.com, or call us at +1 234 000 1111. We're here to help you find the right support.

**Client:** TherapyBot, I just wanted to say thank you so much for everything. Knowing there are options and people who understand... it really means a lot. I think I'm ready to take the next step. Goodbye, and thanks again.

**TherapyRobo**: That's wonderful to hear! It takes courage to move forward, and I'm genuinely happy that you feel ready. Remember, we're here to support you if you need anything in the future. All the best on your journey!

**Note**

> As the Google Gen AI capston project should run smoothly and end to end the responses for the TherapyRobo were generated by gemini in a role of client.If you wanted to use the Therapy Robo and chat with its agent, check the [interctive version](https://www.kaggle.com/code/artamirzadeh/therapyroboinputversion).
> https://www.kaggle.com/code/artamirzadeh/therapyroboinputversion


# **The End**